# Small dataset

In [0]:
from pyspark.sql.window import Window
from pyspark.sql.functions import row_number

dbutils.fs.rm("/mnt/filesystem/delta_antypatterns/facts_and_merge/flights", recurse= True)
windowSpec = Window.orderBy(["date", "origin", "destination"])

df = spark.read.format("csv").option("header", True).option("inferSchema", True).load("/databricks-datasets/flights/departuredelays.csv")
df = df.withColumn("id",row_number().over(windowSpec))
df.write.format("delta").mode("overwrite").partitionBy("origin").save("/mnt/filesystem/delta_antypatterns/facts_and_merge/flights")

### Update 2 partitions

In [0]:
from pyspark.sql.window import Window
from pyspark.sql.functions import row_number
from pyspark.sql.functions import lit

windowSpec = Window.orderBy(["date", "origin", "destination"])

df = spark.read.format("csv").option("header", True).option("inferSchema", True).load("/databricks-datasets/flights/departuredelays.csv")
df = df.withColumn("id",row_number().over(windowSpec))\
      .withColumn("distance", lit(0))\
      .filter("origin IN ('ATL', 'DFW') AND date%2 = 0")
      #.filter("date%2 = 0")
#df.show()
df.createOrReplaceTempView("stg_flights")

+-------+-----+--------+------+-----------+----+
 date|delay|distance|origin|destination| id|
+-------+-----+--------+------+-----------+----+
1010600| -3| 0| ATL| IAH| 185|
1010600| 0| 0| DFW| PHX| 216|
1010630| -5| 0| ATL| MCO| 555|
1010630| -1| 0| ATL| MSP| 556|
1010630| 56| 0| DFW| HOU| 568|
1010640| -4| 0| ATL| MIA| 662|
1010640| -3| 0| DFW| CLT| 668|
1010650| 0| 0| DFW| PHL| 753|
1010700| 9| 0| ATL| BWI| 832|
1010700| 1| 0| ATL| CLT| 833|
1010700| -2| 0| ATL| DFW| 834|
1010700| 18| 0| ATL| LGA| 835|
1010700| -6| 0| DFW| BOS| 865|
1010700| -9| 0| DFW| CVG| 866|
1010700| -1| 0| DFW| DTW| 867|
1010700| -9| 0| DFW| MCI| 868|
1010700| 139| 0| DFW| ORD| 869|
1010700| -5| 0| DFW| STL| 870|
1010710| -1| 0| DFW| ATL|1042|
1010710| -16| 0| DFW| TUL|1043|
+-------+-----+--------+------+-----------+----+
only showing top 20 rows

In [0]:
%sql
--CACHE TABLE stg_flights

In [0]:
%sql
merge into delta.`/mnt/filesystem/delta_antypatterns/facts_and_merge/flights` as tgt
using stg_flights as src 
on src.id = tgt.id and tgt.origin in ('ATL', 'DFW')
WHEN MATCHED 
  THEN UPDATE SET tgt.date = src.date, tgt.delay = src.delay, tgt.distance = src.distance, tgt.origin = src.origin, tgt.destination = src.destination
WHEN NOT MATCHED 
  THEN INSERT *

num_affected_rows,num_updated_rows,num_deleted_rows,num_inserted_rows
729562,729562,0,0


In [0]:
df_part_merge = spark.sql("""
          select * from stg_flights as sf

          union

          select * 
          from delta.`/mnt/filesystem/delta_disjoint_predicates/flights` as f
              left anti join stg_flights as sf
              on sf.id = f.id
          where origin in ('ATL', 'DFW')
      """)

df_part_merge.write\
  .format("delta")\
  .mode("overwrite")\
  .option("replaceWhere", "origin IN ('ATL', 'DFW')")\
  .save("/mnt/filesystem/delta_disjoint_predicates/flights")


### Update all partitions

In [0]:
from pyspark.sql.window import Window
from pyspark.sql.functions import row_number
from pyspark.sql.functions import lit

windowSpec = Window.orderBy(["date", "origin", "destination"])

df = spark.read.format("csv").option("header", True).option("inferSchema", True).load("/databricks-datasets/flights/departuredelays.csv")
df = df.withColumn("id",row_number().over(windowSpec))\
      .withColumn("distance", lit(0))\
      .filter("date%2 = 0")
#df.show()
df.createOrReplaceTempView("stg_flights")

In [0]:
%sql
merge into delta.`/mnt/filesystem/delta_antypatterns/facts_and_merge/flights` as tgt
using stg_flights as src
on src.id = tgt.id
WHEN MATCHED 
  THEN UPDATE SET tgt.date = src.date, tgt.delay = src.delay, tgt.distance = src.distance, tgt.origin = src.origin, tgt.destination = src.destination
WHEN NOT MATCHED 
  THEN INSERT *

In [0]:
df_part_merge = spark.sql("""
          select * from stg_flights as sf

          union

          select * 
          from delta.`/mnt/filesystem/delta_disjoint_predicates/flights` as f
              left anti join stg_flights as sf
              on sf.id = f.id
      """)

df_part_merge.write\
  .format("delta")\
  .mode("overwrite")\
  .option("replaceWhere", "origin IN ('ATL', 'DFW')")\
  .save("/mnt/filesystem/delta_disjoint_predicates/flights")


# Bigger dataset (x100)

In [0]:
from pyspark.sql.window import Window
from pyspark.sql.functions import row_number

dbutils.fs.rm("/mnt/filesystem/delta_antypatterns/facts_and_merge/flights_100", recurse= True)
windowSpec = Window.orderBy(["date", "origin", "destination"])

df = spark.read.format("csv").option("header", True).option("inferSchema", True).load("/databricks-datasets/flights/departuredelays.csv")

max_rn = 0
for i in range(1,100):
  df = df.withColumn("id",row_number().over(windowSpec) + max_rn)
  df.write.format("delta").mode("append").partitionBy("origin").save("/mnt/filesystem/delta_antypatterns/facts_and_merge/flights_100")
  max_rn = df.count()*i

### Update 2 partitions

In [0]:
from pyspark.sql.window import Window
from pyspark.sql.functions import row_number
from pyspark.sql.functions import lit

windowSpec = Window.orderBy(["date", "origin", "destination"])

df = spark.read.format("csv").option("header", True).option("inferSchema", True).load("/databricks-datasets/flights/departuredelays.csv")
df = df.withColumn("id",row_number().over(windowSpec))\
      .withColumn("distance", lit(0))\
      .filter("origin IN ('ATL', 'DFW') AND date%2 = 0")
      #.filter("date%2 = 0")
#df.show()
df.createOrReplaceTempView("stg_flights")

In [0]:
%sql
merge into delta.`/mnt/filesystem/delta_antypatterns/facts_and_merge/flights_100` as tgt
using stg_flights as src
on src.id = tgt.id and tgt.origin in ('ATL', 'DFW')
WHEN MATCHED 
  THEN UPDATE SET tgt.date = src.date, tgt.delay = src.delay, tgt.distance = src.distance, tgt.origin = src.origin, tgt.destination = src.destination
WHEN NOT MATCHED 
  THEN INSERT *

num_affected_rows,num_updated_rows,num_deleted_rows,num_inserted_rows
82196,82196,0,0


In [0]:
df_part_merge = spark.sql("""
          select * from stg_flights as sf

          union all

          select * from delta.`/mnt/filesystem/delta_antypatterns/facts_and_merge/flights_100` as f
            left anti join stg_flights as sf
            on sf.id = f.id
          where origin in ('ATL', 'DFW')
      """)

df_part_merge.write\
  .format("delta")\
  .mode("overwrite")\
  .option("replaceWhere", "origin IN ('ATL', 'DFW')")\
  .save("/mnt/filesystem/delta_antypatterns/facts_and_merge/flights_100")

### Update all partitions

In [0]:
from pyspark.sql.window import Window
from pyspark.sql.functions import row_number
from pyspark.sql.functions import lit

windowSpec = Window.orderBy(["date", "origin", "destination"])

df = spark.read.format("csv").option("header", True).option("inferSchema", True).load("/databricks-datasets/flights/departuredelays.csv")
df = df.withColumn("id",row_number().over(windowSpec))\
      .withColumn("distance", lit(0))\
      .filter("date%2 = 0")
df.createOrReplaceTempView("stg_flights")

+-------+-----+--------+------+-----------+---+
 date|delay|distance|origin|destination| id|
+-------+-----+--------+------+-----------+---+
1010010| -6| 0| SEA| CLT| 2|
1010020| -2| 0| SFO| CLT| 3|
1010020| 0| 0| SFO| DFW| 4|
1010030| -8| 0| LAS| ATL| 9|
1010030| -2| 0| PDX| CLT| 10|
1010030| -7| 0| SFO| PHL| 11|
1010040| -6| 0| SLC| ATL| 14|
1010050| -6| 0| ANC| PDX| 17|
1010110| 7| 0| LAX| IAH| 21|
1010110| -6| 0| PHX| DCA| 22|
1010120| -10| 0| FAI| SEA| 23|
1010120| 2| 0| LAS| CLT| 24|
1010120| -4| 0| LAX| DFW| 25|
1010150| -15| 0| ANC| PHX| 28|
1010154| -7| 0| SJU| EWR| 29|
1010206| -4| 0| SJU| JFK| 30|
1010344| -7| 0| SJU| BOS| 35|
1010420| -4| 0| SJU| MCO| 37|
1010422| -2| 0| SJU| BDL| 38|
1010500| -5| 0| HNL| ITO| 39|
+-------+-----+--------+------+-----------+---+
only showing top 20 rows

In [0]:
%sql
merge into delta.`/mnt/filesystem/delta_antypatterns/facts_and_merge/flights_100` as tgt
using stg_flights as src
on src.id = tgt.id
WHEN MATCHED 
  THEN UPDATE SET tgt.date = src.date, tgt.delay = src.delay, tgt.distance = src.distance, tgt.origin = src.origin, tgt.destination = src.destination
WHEN NOT MATCHED 
  THEN INSERT *

num_affected_rows,num_updated_rows,num_deleted_rows,num_inserted_rows
729562,729562,0,0


In [0]:
df_part_merge = spark.sql("""
          select * from stg_flights as sf

          union all

          select * from delta.`/mnt/filesystem/delta_antypatterns/facts_and_merge/flights_100` as f
            left anti join stg_flights as sf
            on sf.id = f.id
      """)

df_part_merge.write\
  .format("delta")\
  .mode("overwrite")\
  .save("/mnt/filesystem/delta_antypatterns/facts_and_merge/flights_100")